# Azure OpenAI - Accessing files on protected Azure storage
Jupyter notebook to demo the use of Azure OpenAI with an Azure Storage that has its access restricted.

In [1]:
# Importing required packages
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import base64
import os

In [2]:
# Extracting environment variables
AOAI_API_BASE = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")

In [3]:
# Defining storage, container and blob variables
STORAGE_URL = "https://<STORAGE_ACCOUNT_NAME>.blob.core.windows.net"
STORAGE_CONTAINER = "<STORAGE_CONTAINER_NAME>"
STORAGE_BLOB = "<STORAGE_BLOB_NAME>"

In [4]:
# Defining helper function to authenticate with Azure Blob Storage
def get_blob_service_client(storage_url):
    print(f"Step 1 - Authenticating with Azure Blob Storage: {storage_url}.")
    credential = DefaultAzureCredential()
    blob_service_client = BlobServiceClient(
        storage_url,
        credential=credential
    )

    return blob_service_client

In [5]:
# Defining helper function to get blob content
def get_blob_to_base64(blob_service_client: BlobServiceClient, container_name, blob_name):
    print(f"Step 2 - Downloading blob: {blob_name}.")
    blob_client = blob_service_client.get_blob_client(
        container=container_name,
        blob=blob_name
    )
    downloader = blob_client.download_blob(
        max_concurrency=1
    )
    blob_content = downloader.readall()
    blob_base64 = base64.b64encode(blob_content).decode('utf-8')

    return blob_base64

In [6]:
# Defining helper function to call Azure OpenAI endpoint using Python SDK
def analyse_blob(base64_image):
    print(f"Step 3 - Analysing image using Azure OpenAI.")

    client = AzureOpenAI(
        azure_endpoint = AOAI_API_BASE,
        api_key = AOAI_API_KEY,
        api_version = AOAI_API_VERSION
    )

    response = client.chat.completions.create(
        model = AOAI_DEPLOYMENT,
        messages = [
            {"role": "system", "content": "You are a useful image analyser."},
            {"role": "user", "content": [  
                { 
                    "type": "text", 
                    "text": "Please, describe the image." 
                },
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                }
            ]} 
        ]
    )
    print("--------------------")
    print(f"Results: {response.choices[0].message.content}")
    return response

In [7]:
# Step 1: Authenticate with Azure Blob Storage
blob_service_client = get_blob_service_client(
    storage_url=STORAGE_URL
)

# Step 2: Get blob content in Base64 format
base64_image = get_blob_to_base64(
    blob_service_client=blob_service_client,
    container_name=STORAGE_CONTAINER,
    blob_name=STORAGE_BLOB
)

# Step 3: Call Azure OpenAI endpoint using Python SDK
response = analyse_blob(
    base64_image=base64_image
)

Step 1 - Authenticating with Azure Blob Storage: https://lazizaoaistorage.blob.core.windows.net.
Step 2 - Downloading blob: mslogo.png.
Step 3 - Analysing image using Azure OpenAI.
--------------------
Results: The image features a simple grid composed of four colored squares. The top left square is orange, the top right square is green, the bottom left square is blue, and the bottom right square is yellow. The squares are arranged in a 2x2 format.
